In [1]:
import codecs
import networkx as nx
import pandas as pd
import pickle
from os import listdir
from os.path import isfile, join
from networkx.algorithms import isomorphism
from networkx.drawing.nx_pydot import write_dot
from utils import printProgressBar

In [ ]:
# glawinette column number
lemma1 = 0
lemma2 = 1
cat1 = 2
cat2 = 3
origine_morpho = 4
origine_def = 5
BAP1 = 6
BAP2 = 7
BAPsize = 8
FAP1 = 9
FAP2 = 10
FAPsize = 11
radical = 12
FAPtype = 13

# demonette column number
graph_1 = 3
graph_2 = 6
cat_1 = 8
cat_2 = 10
cstr_1 = 14
cstr_2 = 17
complexite = 19
orientation = 21
fichier_origine = 43

def FAPconverter(input_fap):
    return input_fap.replace('(.+)', 'X').replace('$', '').replace('^', '')

def category_shortening(cat):
    if cat != 'Num' and cat[0] == 'N':
        if cat[1] == 'p':  # nom propre
            return 'Np'
        return 'N'  # nom
    return cat

header = ''
glawi_dict = dict()
with codecs.open('glawinette-series.csv', 'r', encoding='utf-8') as f:
    for line_num, line in enumerate(f):
        if line_num >= 1:
            elements = line.replace('\n','').replace(' ','').split('\t')
            glawi_dict[(elements[lemma1], elements[lemma2])] = FAPconverter(elements[FAP1]) + '-' + FAPconverter(elements[FAP2])
            glawi_dict[(elements[lemma2], elements[lemma1])] = FAPconverter(elements[FAP2]) + '-' + FAPconverter(elements[FAP1])
print(len(glawi_dict))

# create binary files for graph

In [ ]:
frcowvec_categories = {'Nm': 'NOM', 'Nf': 'NOM', 'Nmp': 'NOM', 'Nfp': 'NOM', 'Nx': 'NOM', 'More': 'NOM',
                       'Npm': 'NAM', 'Npf': 'NAM', 'Npx': 'NAM', 'Npmp': 'NAM', 'Npfp': 'NAM',
                       'Adj': 'ADJ', 'V': 'VER', 'Num': 'NUM', 'Pro': 'PRO', 'Adv': 'ADV'}

def frcowvec_cat_conversion(lexeme):
    old_cat = lexeme.split('_')[-1]
    new_cat = frcowvec_categories.get(old_cat)
    return lexeme.split('_')[0] + '_' + new_cat

frequencies = pd.read_csv('frequencies-frcowvec-filtered.csv', header=0, index_col=0)
frequencies.shape

In [ ]:
input_dir = 'demonette-glawinette_families'
input_files = [f for f in listdir(input_dir) if isfile(join(input_dir, f))]
output_dir = 'demonette-glawinette_graph_binary'

for input_file in input_files:
    fam_id = input_file.split()[0]
    group_id = fam_id.split('-')[0]
    H = nx.DiGraph()
    with codecs.open(join(input_dir, input_file), 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f):
            if line_num >= 2:
                elements = line.replace('\n','').replace(' ','').split('\t')
                va = elements[graph_1] + '_' + elements[cat_1]
                vb = elements[graph_2] + '_' + elements[cat_2]
                if H.has_edge(va, vb) or H.has_edge(vb, va):
                    continue
                try:
                    freq_a = frequencies.loc[frcowvec_cat_conversion(va)]['freq']
                except KeyError:
                    freq_a = 0
                try:
                    freq_b = frequencies.loc[frcowvec_cat_conversion(vb)]['freq']
                except KeyError:
                    freq_b = 0
                H.add_node(va, label=category_shortening(elements[cat_1]), frequency=freq_a)
                H.add_node(vb, label=category_shortening(elements[cat_2]), frequency=freq_b)
                if elements[orientation] == 'as2de' or elements[orientation] == 'as2des':
                    if (elements[graph_1], elements[graph_2]) in glawi_dict.keys():
                        H.add_edge(va, vb, label=elements[cstr_1] + '-' + elements[cstr_2]\
                                  + '$' + glawi_dict.get((elements[graph_1], elements[graph_2])))
                    else:
                        H.add_edge(va, vb, label=elements[cstr_1] + '-' + elements[cstr_2])
                elif elements[orientation] == 'de2as' or elements[orientation] == 'des2as':
                    if (elements[graph_2], elements[graph_1]) in glawi_dict.keys():
                        H.add_edge(vb, va, label=elements[cstr_2] + '-' + elements[cstr_1]\
                                  + '$' + glawi_dict.get((elements[graph_2], elements[graph_1])))
                    else:
                        H.add_edge(vb, va, label=elements[cstr_2] + '-' + elements[cstr_1])
                else:
                    if (elements[graph_1], elements[graph_2]) in glawi_dict.keys():
                        H.add_edge(va, vb, label=elements[cstr_1] + '-' + elements[cstr_2] + '_' + elements[orientation]\
                                  + '$' + glawi_dict.get((elements[graph_1], elements[graph_2])))
                        H.add_edge(vb, va, label=elements[cstr_2] + '-' + elements[cstr_1] + '_' + elements[orientation]\
                                  + '$' + glawi_dict.get((elements[graph_2], elements[graph_1])))
                    else:
                        H.add_edge(va, vb, label=elements[cstr_1] + '-' + elements[cstr_2] + '_' + elements[orientation])
                        H.add_edge(vb, va, label=elements[cstr_2] + '-' + elements[cstr_1] + '_' + elements[orientation])
    graph_file = open(join(output_dir, fam_id), 'wb')
    pickle.dump(H, graph_file)
    graph_file.close()
    print(input_file.split()[0], end='\r')

# creation of formal context

In [2]:
input_dir = 'demonette-glawinette_graph_binary'
input_files = [f for f in listdir(input_dir) if isfile(join(input_dir, f))]
input_files.sort()
node_count_dict = dict()
for graph in input_files:
    G2 = pickle.load(open(join(input_dir, graph), 'rb'))
    node_count_dict[graph] = len(G2)
    
subgroup_prev = ''
context = pd.DataFrame()
counter = 1
input_files_count = len(input_files)

In [3]:
def edge_compare(e1, e2):
    if '$' in e1['label'] and '$' not in e2['label']:
        return e1['label'].split('$')[0] == e2['label']
    else:
        return e1['label'] == e2['label']

for subgraph in input_files:
    counter += 1
    subgroup_id = subgraph.split('-')[0]
    if subgroup_id == subgroup_prev:
        continue
    membership = [0] * len(input_files)
    G2 = pickle.load(open(join(input_dir, subgraph), 'rb'))
    G2_node_count = node_count_dict.get(subgraph)
    supergroup_prev = ''
    is_subgraph = 0
    for counter2 in range(input_files_count-1, -1, -1):
        if membership[counter2] == 1:
            continue
        supergraph = input_files[counter2]
        supergroup_id = supergraph.split('-')[0]
        if supergroup_id == subgroup_id:
            membership[counter2] = 1
            continue
        if supergroup_id == supergroup_prev:
            membership[counter2] = membership[counter2+1]
            continue
        print(subgraph + ' ' + supergraph, end='\r')
        G1_node_count = node_count_dict.get(supergraph)
        if G1_node_count > G2_node_count:
            G1 = pickle.load(open(join(input_dir, supergraph), 'rb'))
            GM = isomorphism.DiGraphMatcher(G1, G2, node_match=lambda v1,v2: v1['label'] == v2['label'], edge_match=edge_compare)
            if GM.subgraph_is_isomorphic():
                membership[counter2] = 1
                try:
                    membership_of_G1 = context[supergroup_id.replace('F', 'G')]
                    membership = (membership_of_G1 | membership).astype(int)
                except KeyError:
                    pass
        supergroup_prev = supergroup_id
    subgroup_prev = subgroup_id
    membership = pd.Series(membership, name=subgroup_id.replace('F', 'G'))
    context = pd.concat([context, membership], axis=1)
context.index = input_files
print(context.shape)

(2933, 168)F00000-105


In [4]:
context.to_csv('DG-context.csv')